# Random Forest

Импортирт библиотек

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [3]:
import joblib

Загрузка данных

In [4]:
X_train = pd.read_csv('DATA\\train_data_19234_62.zip', compression='zip')

In [5]:
X_train = X_train.set_index(X_train.user_id)
X_train = X_train.drop('user_id', axis=1)

In [6]:
y_train = pd.read_csv('DATA\\target_feature.csv')

In [7]:
y_train = y_train.set_index(y_train.user_id)
y_train = y_train.drop('user_id', axis=1)

In [8]:
y_train = y_train.is_gone

Настройка параметров классификатора перебором по сетке

In [9]:
model = RandomForestClassifier(random_state=100)

In [10]:
parametrs = {'n_estimators': range(100, 1001, 10)
             'max_features': range(1, 63),
             'criterion': ['entropy', 'gini'],
             'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, None]
             'min_samples_split': range(1, 15),
             'min_samples_leaf': range(1, 15)}

In [11]:
search = GridSearchCV(model, parametrs, scoring='roc_auc', cv=4, n_jobs=-1)

In [12]:
search.fit(X_train, y_train)# ждем около 9 часов

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=100),
             n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [7],
                         'max_features': [34], 'min_samples_leaf': [1],
                         'min_samples_split': [5], 'n_estimators': [100]},
             scoring='roc_auc')

In [13]:
best_model = search.best_estimator_

In [14]:
best_model

RandomForestClassifier(criterion='entropy', max_depth=7, max_features=34,
                       min_samples_split=5, random_state=100)

Cохраняем настроенную модель

In [15]:
#joblib.dump(best_model, "models\\best_model.pkl") 

In [16]:
# загрузка сохраненной модели
best_model = joblib.load("models\\best_model.pkl")

Проверка кроссвалидацией

In [17]:
model_cv = RandomForestClassifier(**best_model.get_params())

In [18]:
cv_scores = cross_val_score(model_cv, X_train, y_train, scoring='roc_auc', cv=5, n_jobs=-1)

In [19]:
mean_cv_scores = np.mean(cv_scores)
print ('mean ROC_AUC score', mean_cv_scores)

mean ROC_AUC score 0.8949098178804634


Оценка важности признаков

In [20]:
feature_importances = best_model.feature_importances_

In [21]:
feature_importances_df = pd.DataFrame({'features': X_train.columns,
                                       'feature_importances': feature_importances})

In [22]:
feature_importances_df.sort_values('feature_importances', ascending=False) \
                      .reset_index(drop=True).head(10)

,features,feature_importances
0,c_uniq*S_steps_ratio,0.300296
1,(c_freq[i]/c_freq[i+1])*(c_uniq*S_steps_ratio),0.189927
2,c_uniq,0.178980
3,c_uniq_ratio,0.060311
4,c_count_ratio,0.034709
5,c_freq,0.034403
6,c_density,0.021664
7,koef_day_1,0.012234
8,s_count_ratio,0.011037
9,steps_ratio,0.011000


Формируем и сохраняем файл с прогнозом

In [23]:
X_test = pd.read_csv('DATA\\test_data_6184_62.zip', compression='zip')

In [24]:
submit = X_test.user_id.to_frame()

In [25]:
X_test = X_test.set_index(X_test.user_id)
X_test = X_test.drop('user_id', axis=1)

In [26]:
y_pred = best_model.predict_proba(X_test)

In [27]:
submit['is_gone'] = best_model.predict_proba(X_test)[:, 1]

In [28]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6184 entries, 0 to 6183
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  6184 non-null   int64  
 1   is_gone  6184 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 96.7 KB


In [29]:
assert submit.user_id.nunique() == X_test.index.nunique()# проверяем

In [30]:
submit.to_csv('DATA\\SUBMISSIONS\\RF_100_34_7_feature_62.csv', index=False)# сохраняем 